In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob, time, datetime, re

In [79]:
files = glob.glob("../data/springfield_springfield_all/"+"*.csv")
shows = pd.concat([pd.read_csv(file, lineterminator='\n') for file in files])

In [80]:
shows.shape

(117375, 3)

In [81]:
shows.head()

,text,episode_name,show_name
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,'Allo 'Allo! (1982) s03e01 Episode Script
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,'Allo 'Allo! (1982) s03e02 Episode Script
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,'Allo 'Allo! (1982) s03e03 Episode Script
3,It is possible that\r the more curious among y...,The Flight of Fancy,'Allo 'Allo! (1982) s03e04 Episode Script
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,'Allo 'Allo! (1982) s03e05 Episode Script


In [82]:
shows.tail()

,text,episode_name,show_name
42093,: Out of the night When the full moon ls brigh...,The Flaming Arrow,Zorro (1957) s02e11 Episode Script
42094,: Out of the night When the full moon ls brigh...,Zorro Fights a Duel,Zorro (1957) s02e12 Episode Script
42095,: Out of the night When the full moon ls brigh...,Amnesty for Zorro,Zorro (1957) s02e13 Episode Script
42096,: Out of the night When the full moon ls brigh...,The Runaways,Zorro (1957) s02e14 Episode Script
42097,: Out of the night When the full moon ls brigh...,The Iron Box,Zorro (1957) s02e15 Episode Script


In [83]:
shows.describe()

,text,episode_name,show_name
count,117225,117371,117374
unique,116735,88890,117315
top,"1\r Happy New Year 2015 - New Year, New Color ...",Episode 1,The A-Team (1983) Episode Scripts
freq,4,893,15


In [84]:
shows.isnull().sum()

text            150
episode_name      4
show_name         1
dtype: int64

In [85]:
shows[shows["text"].isnull()]["show_name"].value_counts()

The A-Team (1983) Episode Scripts                             15
Shaun the Sheep (2007) Episode Scripts                         9
TV Show Episode Scripts                                        3
COPS (1989) s28e07 Episode Script                              1
1000 Ways to Die s04e13 Episode Script                         1
                                                              ..
COPS (1989) s28e18 Episode Script                              1
COPS (1989) s27e20 Episode Script                              1
Agatha Christie's Partners in Crime (1983) Episode Scripts     1
1000 Ways to Die s05e11 Episode Script                         1
COPS (1989) s26e16 Episode Script                              1
Name: show_name, Length: 125, dtype: int64

#### Cleaning Show Name Column

There is one row with no information. Will drop then clean remaining items in show_name column

In [86]:
shows[shows["show_name"].isnull()].shape
shows = shows.drop(shows[shows["show_name"].isnull()].index)

In [87]:
shows["show_name"] = shows["show_name"].str.replace("Episode Script", "")

In [90]:
shows[shows["show_name"].str.contains(r"s\d{2}")]

,text,episode_name,show_name
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,'Allo 'Allo! (1982) s03e01
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,'Allo 'Allo! (1982) s03e02
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,'Allo 'Allo! (1982) s03e03
3,It is possible that\r the more curious among y...,The Flight of Fancy,'Allo 'Allo! (1982) s03e04
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,'Allo 'Allo! (1982) s03e05
...,...,...,...
42093,: Out of the night When the full moon ls brigh...,The Flaming Arrow,Zorro (1957) s02e11
42094,: Out of the night When the full moon ls brigh...,Zorro Fights a Duel,Zorro (1957) s02e12
42095,: Out of the night When the full moon ls brigh...,Amnesty for Zorro,Zorro (1957) s02e13
42096,: Out of the night When the full moon ls brigh...,The Runaways,Zorro (1957) s02e14


In [101]:
for show in shows["show_name"]:
    if re.match(r"\d{2}$", show):
        print("true")

In [93]:
for show in shows["show_name"]:
    if re.match(r"s\d{2}", show):
    #if shows["show_name"].str.contains(r"s\d{2}"):
        show["season_split"] = show["show_name"].str.rsplit("s")
        show["season_episode"] = show["season_split"].str.get(1)
        show["season_episode_split"] = show["season_episode"].str.split("e")
        show["season"] = show["season_episode_split"].str.get(0)
        show["episode_number"] = show["season_episode_split"].str.get(1)
        show["show_name_year"] = show["season_split"].str.get(0)

        if shows["show_name_year"].str.contains(r"\(\d{4}"):
            shows["show_name_year_split"] = shows["show_name_year"].str.rsplit("(")
            shows["year"] = shows["show_name_year_split"].str.get(1).str.replace(")", "")
            shows["show_name"] = shows["show_name_year_split"].str.get(0).str.replace(",", "").strip("")
        else:
            pass
    else:
        pass

In [94]:
shows.head()

,text,episode_name,show_name
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,'Allo 'Allo! (1982) s03e01
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,'Allo 'Allo! (1982) s03e02
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,'Allo 'Allo! (1982) s03e03
3,It is possible that\r the more curious among y...,The Flight of Fancy,'Allo 'Allo! (1982) s03e04
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,'Allo 'Allo! (1982) s03e05


In [11]:
shows["show_name"] = shows["show_name_split"].str.get(0)

In [12]:
shows.head()

,text,episode_name,show_name,show_name_split
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e01 ]"
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e02 ]"
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e03 ]"
3,It is possible that\r the more curious among y...,The Flight of Fancy,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e04 ]"
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e05 ]"


In [13]:
shows["year_season_episode"] = shows["show_name_split"].str.get(1)

In [14]:
shows.head()

,text,episode_name,show_name,show_name_split,year_season_episode
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e01 ]",1982) s03e01
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e02 ]",1982) s03e02
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e03 ]",1982) s03e03
3,It is possible that\r the more curious among y...,The Flight of Fancy,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e04 ]",1982) s03e04
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e05 ]",1982) s03e05


In [15]:
shows["year_season_episode_split"] = shows["year_season_episode"].str.split(" ")

In [16]:
shows["year"] = shows["year_season_episode_split"].str.get(0).str.replace(")", "")

In [17]:
shows.head()

,text,episode_name,show_name,show_name_split,year_season_episode,year_season_episode_split,year
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e01 ]",1982) s03e01,"[1982), s03e01, ]",1982
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e02 ]",1982) s03e02,"[1982), s03e02, ]",1982
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e03 ]",1982) s03e03,"[1982), s03e03, ]",1982
3,It is possible that\r the more curious among y...,The Flight of Fancy,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e04 ]",1982) s03e04,"[1982), s03e04, ]",1982
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e05 ]",1982) s03e05,"[1982), s03e05, ]",1982


In [18]:
shows["season_episode_split"] = shows["year_season_episode_split"].str.get(1).str.split("e")

In [19]:
shows.head()

,text,episode_name,show_name,show_name_split,year_season_episode,year_season_episode_split,year,season_episode_split
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e01 ]",1982) s03e01,"[1982), s03e01, ]",1982,"[s03, 01]"
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e02 ]",1982) s03e02,"[1982), s03e02, ]",1982,"[s03, 02]"
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e03 ]",1982) s03e03,"[1982), s03e03, ]",1982,"[s03, 03]"
3,It is possible that\r the more curious among y...,The Flight of Fancy,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e04 ]",1982) s03e04,"[1982), s03e04, ]",1982,"[s03, 04]"
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e05 ]",1982) s03e05,"[1982), s03e05, ]",1982,"[s03, 05]"


In [20]:
shows["season"] = shows["season_episode_split"].str.get(0).str.replace("s", "")

In [21]:
shows["episode_number"] = shows["season_episode_split"].str.get(1)

In [22]:
shows.head()

,text,episode_name,show_name,show_name_split,year_season_episode,year_season_episode_split,year,season_episode_split,season,episode_number
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e01 ]",1982) s03e01,"[1982), s03e01, ]",1982,"[s03, 01]",03,01
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e02 ]",1982) s03e02,"[1982), s03e02, ]",1982,"[s03, 02]",03,02
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e03 ]",1982) s03e03,"[1982), s03e03, ]",1982,"[s03, 03]",03,03
3,It is possible that\r the more curious among y...,The Flight of Fancy,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e04 ]",1982) s03e04,"[1982), s03e04, ]",1982,"[s03, 04]",03,04
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,'Allo 'Allo!,"['Allo 'Allo! , 1982) s03e05 ]",1982) s03e05,"[1982), s03e05, ]",1982,"[s03, 05]",03,05


In [23]:
shows = shows.drop(columns=["show_name_split", "year_season_episode", "year_season_episode_split", "season_episode_split"])

In [24]:
shows.head()

,text,episode_name,show_name,year,season,episode_number
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,'Allo 'Allo!,1982,03,01
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,'Allo 'Allo!,1982,03,02
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,'Allo 'Allo!,1982,03,03
3,It is possible that\r the more curious among y...,The Flight of Fancy,'Allo 'Allo!,1982,03,04
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,'Allo 'Allo!,1982,03,05


In [25]:
shows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117375 entries, 0 to 42097
Data columns (total 6 columns):
text              117225 non-null object
episode_name      117371 non-null object
show_name         117374 non-null object
year              72418 non-null object
season            72418 non-null object
episode_number    71372 non-null object
dtypes: object(6)
memory usage: 6.3+ MB


#### Cleaning the Show Name Column 

There is one row that has no show name, and when inspected, is a completely null row. Will drop

(1, 6)

In [28]:
shows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117373 entries, 0 to 42097
Data columns (total 6 columns):
text              117224 non-null object
episode_name      117370 non-null object
show_name         117373 non-null object
year              72417 non-null object
season            72417 non-null object
episode_number    71371 non-null object
dtypes: object(6)
memory usage: 6.3+ MB


In [29]:
shows["show_name"].value_counts()

Doctor Who                                    702
Naruto: Shippuden                             500
Law & Order                                   456
Teenage Mutant Ninja Turtles                  315
Last of the Summer Wine                       287
                                             ... 
Richard Hammond's Invisible Worlds s01e01       1
Everybody Loves Raymond s02e13                  1
Rizzoli and Isles s05e11                        1
The Simpsons s05e20                             1
The Dead Zone s04e02                            1
Name: show_name, Length: 48869, dtype: int64

The NaN values in the year, season, and episode_number are due to a different show_name format. Will split the dataframe to clean these show names, fill in the missing values with the ones in the show names and then recombine with the already cleaned data 

In [30]:
#pulling out show_names with a number
mask = shows["show_name"].str.contains("\d")

In [31]:
shows[mask]

,text,episode_name,show_name,year,season,episode_number
12,I'd like to marry a girl that's skinny. I thin...,Pilot,'Til Death s01e01,NaN,NaN,NaN
13,"* All Right! * Hey! * Oh, oh, oh, oh, oh, oh! ...",Sex for Furniture,'Til Death s01e02,NaN,NaN,NaN
14,"Oh, I took way too much hand cream. Here. What...",The Ring,'Til Death s01e03,NaN,NaN,NaN
15,"* All right! * Hey! * Oh, oh, oh, oh, oh, oh! ...",The Wood Pile,'Til Death s01e04,NaN,NaN,NaN
16,"* All right! * Hey! * Oh, oh, oh, oh, oh, oh! ...",The Garage Band,'Til Death s01e05,NaN,NaN,NaN
...,...,...,...,...,...,...
41984,"Previously on ""zero hour""\r I am gonna find\r...",Balance,Zero Hour s01e09,NaN,NaN,NaN
41985,"1\r Previously on ""Zero Hour""\r (Hank) I've s...",Escapement,Zero Hour s01e10,NaN,NaN,NaN
41986,"1\r Previously on ""Zero Hour""\r (Hank)\r They...",Hands,Zero Hour s01e11,NaN,NaN,NaN
41987,"Previously on ""zero hour""\r What am i looking...",Ratchet,Zero Hour s01e12,NaN,NaN,NaN


In [48]:
shows_to_clean = shows[mask]

In [50]:
shows_to_clean.tail()

,text,episode_name,show_name,year,season,episode_number
41984,"Previously on ""zero hour""\r I am gonna find\r...",Balance,Zero Hour s01e09,NaN,NaN,NaN
41985,"1\r Previously on ""Zero Hour""\r (Hank) I've s...",Escapement,Zero Hour s01e10,NaN,NaN,NaN
41986,"1\r Previously on ""Zero Hour""\r (Hank)\r They...",Hands,Zero Hour s01e11,NaN,NaN,NaN
41987,"Previously on ""zero hour""\r What am i looking...",Ratchet,Zero Hour s01e12,NaN,NaN,NaN
41988,"Previously on ""Zero Hour""\r I believe there a...",Spring,Zero Hour s01e13,NaN,NaN,NaN


In [52]:
shows_to_clean["show_name_split"] = shows_to_clean["show_name"].str.split("s")
shows_to_clean["show_name"] = shows_to_clean["show_name_split"].str.get(0)
shows_to_clean["season_episode"] = shows_to_clean["show_name_split"].str.get(1)
shows_to_clean["season_episode_split"] = shows_to_clean["season_episode"].str.split("e")
shows_to_clean["season"] = shows_to_clean["season_episode_split"].str.get(0)
shows_to_clean["episode_number"] = shows_to_clean["season_episode_split"].str.get(1)

/Users/dsi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/dsi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/dsi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [53]:
shows_to_clean.head()

,text,episode_name,show_name,year,season,episode_number,show_name_split,season_episode,season_episode_split
12,I'd like to marry a girl that's skinny. I thin...,Pilot,'Til Death,NaN,01,01,"['Til Death , 01e01 ]",01e01,"[01, 01 ]"
13,"* All Right! * Hey! * Oh, oh, oh, oh, oh, oh! ...",Sex for Furniture,'Til Death,NaN,01,02,"['Til Death , 01e02 ]",01e02,"[01, 02 ]"
14,"Oh, I took way too much hand cream. Here. What...",The Ring,'Til Death,NaN,01,03,"['Til Death , 01e03 ]",01e03,"[01, 03 ]"
15,"* All right! * Hey! * Oh, oh, oh, oh, oh, oh! ...",The Wood Pile,'Til Death,NaN,01,04,"['Til Death , 01e04 ]",01e04,"[01, 04 ]"
16,"* All right! * Hey! * Oh, oh, oh, oh, oh, oh! ...",The Garage Band,'Til Death,NaN,01,05,"['Til Death , 01e05 ]",01e05,"[01, 05 ]"


In [55]:
shows_to_clean.drop(columns=["show_name_split", "season_episode", "season_episode_split"], inplace=True)
shows_to_clean.head()

/Users/dsi/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,text,episode_name,show_name,year,season,episode_number
12,I'd like to marry a girl that's skinny. I thin...,Pilot,'Til Death,NaN,01,01
13,"* All Right! * Hey! * Oh, oh, oh, oh, oh, oh! ...",Sex for Furniture,'Til Death,NaN,01,02
14,"Oh, I took way too much hand cream. Here. What...",The Ring,'Til Death,NaN,01,03
15,"* All right! * Hey! * Oh, oh, oh, oh, oh, oh! ...",The Wood Pile,'Til Death,NaN,01,04
16,"* All right! * Hey! * Oh, oh, oh, oh, oh, oh! ...",The Garage Band,'Til Death,NaN,01,05


In [56]:
shows_to_clean.isnull().sum()

text                 21
episode_name          0
show_name             0
year              44952
season              925
episode_number    15386
dtype: int64

In [58]:
shows_to_clean[shows_to_clean["season"].isnull()]

,text,episode_name,show_name,year,season,episode_number
110,1\r Welcome to 10 O'Clock Live.\r It's Wednes...,"Series 2, Episode 2",10 O'Clock Live,2011,NaN,NaN
111,1\r Welcome to 10 O'Clock Live.\r It's Wedne...,"Series 2, Episode 3",10 O'Clock Live,2011,NaN,NaN
112,1\r Welcome to 10 O'Clock Live.\r It's Wedne...,"Series 2, Episode 4",10 O'Clock Live,2011,NaN,NaN
208,1\r [mysterious music.]\r The day that cha...,The Rabbit Hole,11.22.63,2016,NaN,NaN
209,"Previously on ""11.22.63""\r I need you to go i...",The Kill Floor,11.22.63,2016,NaN,NaN
...,...,...,...,...,...,...
40768,"At number five, we have\r auburn-haired stunne...",A Different Beat,WPC 56,2013,NaN,NaN
40769,1\r Who the hell is this?\r WPC Annie Taylor....,Walk the Line,WPC 56,2013,NaN,NaN
40770,1\r I should have come home earlier\r last nig...,From The Shadows,WPC 56,2013,NaN,NaN
40771,"William Sheppard, I'm arresting you on suspici...",The Wayward Wind,WPC 56,2013,NaN,NaN


In [59]:
shows[mask] = shows_to_clean

In [61]:
shows.head(10)

,text,episode_name,show_name,year,season,episode_number
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,'Allo 'Allo!,1982,03,01
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,'Allo 'Allo!,1982,03,02
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,'Allo 'Allo!,1982,03,03
3,It is possible that\r the more curious among y...,The Flight of Fancy,'Allo 'Allo!,1982,03,04
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,'Allo 'Allo!,1982,03,05
5,Every little breeze\r seems to whisper Louise\...,The Great Un-Escape,'Allo 'Allo!,1982,03,06
6,There cannot be many French\r café owners in a...,Hans Goes Over the Top,'Allo 'Allo!,1982,04,01
7,You are probably wondering what\r I am doing i...,Camp Dance,'Allo 'Allo!,1982,04,02
8,Ah The more observant among you\r may note th...,Good Staff are Hard to Find,'Allo 'Allo!,1982,04,03
9,Things are looking up.\r A slip of the tongue...,The Flying Nun,'Allo 'Allo!,1982,04,04


In [62]:
shows.isnull().sum()

text                149
episode_name          3
show_name             0
year              44956
season              929
episode_number    16436
dtype: int64

In [63]:
len(set(shows["show_name"]))

4613

In [75]:
#remove trailing whitespace
shows["show_name"] = shows["show_name"].str.strip(" ")

In [77]:
set(shows["show_name"])

{'Corner Gas',
 'Con Man',
 'Secret Girlfriend',
 'Merlin',
 '13 Step',
 'Survivorman',
 'Ninjago: Masters of Spinjitzu',
 'Freud',
 'Mind Field',
 'Wellington Paranormal',
 'Hang Ups',
 "Bible's Buried Secrets",
 'Flat TV',
 'Barely Famous',
 'El Dragon: Return of a Warrior',
 'Kingdom Hospital',
 'Fringe',
 'Ground Floor',
 'Romper Stomper',
 'Tarzan and Jane',
 'Viking',
 'Rev.',
 'Murder Most Horrid',
 'In With The Flynns',
 'The Secret',
 'A to Z',
 'Comedians in Cars Getting Coffee',
 'Wasted',
 'Mind Games',
 'Daniel Sloss: Live Shows',
 'The Last Man On Earth',
 'Jackie Chan Adventures',
 'Scooby-Doo Mystery Incorporated',
 'Togetherness',
 'Shower of Stars',
 'Kevin',
 'The Carrie Diaries',
 '60 Minute',
 'Sma',
 'Gone',
 'The Borgia',
 'Sunday Night at the Palladium',
 'This Way Up',
 'SMILF',
 'Mysteries at the Museum',
 '6Degree',
 'Globe Trekker',
 'See',
 'Sex Traffic',
 'The Fall Guy',
 'You Deserve It',
 'Marcella',
 'American Dad',
 'Heroes Reborn',
 'Nature',
 'One Ch

In [70]:
shows["season"].value_counts(ascending=True).head(60)

040              1
-1               1
: Mod            1
 And R           2
ukoi Mon         2
 Ri              2
 To S            2
land             2
 to Victory      2
 Str             2
020              2
hima             2
 Fir             2
 of Lif          2
 2014: In        2
t T              2
 to Sav          2
ualty 1909       2
 a Plan          2
 - Chick         2
0                2
 Down            2
hian             2
h at Work        3
 Food            3
 Invi            3
 W               3
t in Au          4
 Six Wiv         4
33               4
 Whor            4
hing Impo        5
 To Midnight     5
59               5
 End             5
Promi            5
 to L            5
99               5
Riing            5
 Kick            6
: Cod            6
 Po              6
35               6
 Gr              6
: God            6
 S               6
 of Bon          6
: How to R       6
 Engin           6
ion 404          6
Th               6
t Room           6
chanc       

In [45]:
#def clean_show_name_mask(df):
#    df["split_row"] = df["show_name"].str.split("s")
#    df["season_episode"] = df["split_row"].str.get(1)
#    df["show_name"] = df["split_row"].str.get(0)
#    df["season_episode_split"] = df["season_episode"].str.split("e")
#    df["season"] = df["season_episode_split"].str.get(0)
#    df["episode_number"] = df["season_episode_split"].str.get(1)
#    return df["show_name"], df["season"], df["episode_number"]


In [54]:
#shows[mask].apply(clean_show_name_mask, axis=1)